#Base Sintética para o Power BI

In [ ]:
!pip install faker pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker
import numpy as np

# Configurar Faker para nomes em português
fake = Faker('pt_BR')
Faker.seed(42)  # Para reprodutibilidade
random.seed(42)
np.random.seed(42)

# Parâmetros
num_usuarios = 50
num_registros = 300
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 9, 9)

# Listas para geração de dados
generos = ['M'] * 43 + ['F'] * 7  # 43 homens, 7 mulheres
fumantes = ['Sim', 'Não']
marcas_isqueiro = ['Bic', 'Zippo', 'Clipper', 'Flamingo', 'Genérico']
locais = ['Casa', 'Bar', 'Rua', 'Trabalho', 'Casa de Alguém', 'Esquecido em Algum Lugar']
contextos_uso = ['Casa', 'Bar', 'Rua', 'Trabalho', 'Social', 'Festa', 'Rolê']
propositos_uso = ['Cigarro', 'Incenso', 'Vela', 'Fogão', 'Camping']
idades = list(range(18, 61))

# Gerar tabela de usuários
usuarios = []
for i in range(num_usuarios):
    genero = generos[i]
    fumante = random.choices(fumantes, weights=[0.65, 0.35])[0] if genero == 'M' else random.choices(fumantes, weights=[0.35, 0.65])[0]
    data_compra = fake.date_time_between(start_date, end_date - timedelta(days=30))  # Até 30 dias antes do fim
    perdeu = random.choices([True, False], weights=[0.25, 0.75])[0]  # 25% de chance de perda
    data_perda = None
    local_perda = None
    dias_ate_perda = 0
    if perdeu:
        max_perda_date = min(data_compra + timedelta(days=180), end_date)
        data_perda = fake.date_time_between(data_compra + timedelta(days=1), max_perda_date)
        local_perda = random.choices(locais, weights=[0.25, 0.2, 0.3, 0.15, 0.05, 0.05])[0]  # Mais perdas na rua/bar
        dias_ate_perda = (data_perda - data_compra).days
    usuarios.append({
        'id_usuario': i + 1,
        'nome': fake.name(),
        'genero': genero,
        'idade': random.choice(idades),
        'fumante': fumante,
        'marca_isqueiro': random.choice(marcas_isqueiro),
        'data_compra': data_compra,
        'perdeu': perdeu,
        'data_perda': data_perda,
        'local_perda': local_perda,
        'dias_ate_perda': dias_ate_perda
    })

df_usuarios = pd.DataFrame(usuarios)

# Gerar tabela de usos
usos = []
for _ in range(num_registros):
    usuario = random.choice(df_usuarios.to_dict('records'))
    # Data do uso: entre data_compra e data_perda (se perdeu) ou end_date
    end_uso = usuario['data_perda'] if usuario['perdeu'] else end_date
    data_uso = fake.date_time_between(usuario['data_compra'], end_uso)
    contexto = random.choice(contextos_uso)
    # Fumantes usam mais para cigarro, não fumantes para incenso/vela/fogão
    proposito = random.choices(propositos_uso, weights=[0.6, 0.1, 0.1, 0.1, 0.1])[0] if usuario['fumante'] == 'Sim' else random.choices(propositos_uso, weights=[0.15, 0.3, 0.3, 0.2, 0.05])[0]
    # Mais acendimentos para cigarro, menos para outros propósitos
    num_acendimentos = random.randint(5, 50) if proposito == 'Cigarro' else random.randint(1, 20)
    usos.append({
        'id_usuario': usuario['id_usuario'],
        'nome': usuario['nome'],
        'genero': usuario['genero'],
        'idade': usuario['idade'],
        'fumante': usuario['fumante'],
        'marca_isqueiro': usuario['marca_isqueiro'],
        'data_uso': data_uso,
        'contexto_uso': contexto,
        'proposito_uso': proposito,
        'num_acendimentos': num_acendimentos,
        'data_compra': usuario['data_compra'],
        'perdeu': usuario['perdeu'],
        'data_perda': usuario['data_perda'],
        'local_perda': usuario['local_perda'],
        'dias_ate_perda': usuario['dias_ate_perda']
    })

df_usos = pd.DataFrame(usos)

# Adicionar faixa etária
df_usos['faixa_idade'] = pd.cut(df_usos['idade'], bins=[0, 25, 35, 60], labels=['18-25', '26-35', '36+'])

# Adicionar mês/ano para análise temporal
df_usos['mes_ano'] = df_usos['data_uso'].dt.to_period('M')

# Preencher possíveis valores nulos
df_usos = df_usos.fillna({
    'num_acendimentos': 0,
    'dias_ate_perda': 0,
    'local_perda': 'N/A',
    'contexto_uso': 'N/A',
    'proposito_uso': 'N/A',
    'faixa_idade': '36+',
    'mes_ano': 'N/A'
})

# Garantir tipos de dados
df_usos = df_usos.astype({
    'id_usuario': 'int',
    'nome': 'str',
    'genero': 'str',
    'idade': 'int',
    'fumante': 'str',
    'marca_isqueiro': 'str',
    'data_uso': 'datetime64[ns]',
    'contexto_uso': 'str',
    'proposito_uso': 'str',
    'num_acendimentos': 'int',
    'data_compra': 'datetime64[ns]',
    'perdeu': 'bool',
    'data_perda': 'datetime64[ns]',
    'local_perda': 'str',
    'dias_ate_perda': 'int',
    'faixa_idade': 'category',
    'mes_ano': 'str'
})

# Exportar para CSV (pronto para Power BI)
df_usos.to_csv('base_isqueiros_powerbi.csv', index=False, encoding='utf-8')

# Resumo da base
print("=== BASE SINTÉTICA PARA POWER BI (primeiros 10 registros) ===")
print(df_usos.head(10).to_string(index=False))
print(f"\nShape: {df_usos.shape}")
print("Colunas: ", df_usos.columns.tolist())
print(f"NaNs totais: {df_usos.isnull().sum().sum()}")
print(f"Período de usos: {df_usos['data_uso'].min()} a {df_usos['data_uso'].max()}")
print(f"Distribuição gênero: {df_usos['genero'].value_counts().to_dict()}")
print(f"Distribuição fumante: {df_usos['fumante'].value_counts().to_dict()}")
print(f"Distribuição contexto de uso: {df_usos['contexto_uso'].value_counts().to_dict()}")
print(f"Distribuição propósito de uso: {df_usos['proposito_uso'].value_counts().to_dict()}")
print(f"Distribuição local de perda: {df_usos[df_usos['perdeu'] == True]['local_perda'].value_counts().to_dict()}")
print(f"Média de dias até a perda: {df_usos[df_usos['perdeu'] == True]['dias_ate_perda'].mean():.2f} dias")
print(f"Total de acendimentos: {df_usos['num_acendimentos'].sum()}")
print(f"Total de perdas: {df_usos['perdeu'].sum()}")
print(f"Total de usuários únicos: {df_usos['id_usuario'].nunique()}")

ModuleNotFoundError: No module named 'faker'

In [ ]:
df = pd.read_csv('base_isqueiros_powerbi.csv')

In [ ]:
df['data_perda']

,data_perda
0,NaT
1,2025-09-06 00:30:16.890375
2,NaT
3,NaT
4,2025-06-24 01:49:09.872183
...,...
295,2025-05-16 07:07:32.987381
296,NaT
297,NaT
298,2025-09-06 00:30:16.890375
